In [0]:
df_nuevos = spark.read.parquet("/Volumes/workspace/default/retail_lakehouse/Online_Retail_Parquet")

from delta.tables import DeltaTable
from pyspark.sql.functions import col

ruta_bronze = "/Volumes/workspace/default/retail_lakehouse/bronze"
delta_bronze = DeltaTable.forPath(spark, ruta_bronze)
df_bronze_actual = delta_bronze.toDF()

df_nuevos_unicos = df_nuevos.alias("nuevos").join(
    df_bronze_actual.alias("existente"),
    on=[
        col("nuevos.InvoiceNo") == col("existente.InvoiceNo"),
        col("nuevos.StockCode") == col("existente.StockCode")
    ],
    how="left_anti"
)

print(f"🆕 Registros nuevos detectados: {df_nuevos_unicos.count()}")

df_nuevos_unicos.write.format("delta").mode("append").save(ruta_bronze)

spark.read.format("delta").load(ruta_bronze).createOrReplaceTempView("bronze_online_retail") 